# Maximisation couple moyen

Ce notebook propose d'optimiser le rotor d'une machine synchro-réluctante sans aimant à une paire de pôle en vue de maximiser le couple moyen.
Dans ce cas particulier, le couple moyen est directement relié à la différence de perméance magnétique du rotor dans l'axe direct d et l'axe en quadrature q.

Le problème étudié est donc la maximisation de cette différence de perméance via la différence de compliance magnétique. Il est possible de démontrer que ce problème est mal posé.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry

importing NGSolve-6.2.2105-9-g5a835126f


## 1) Définition du maillage

La première étape consiste à définir une géométrie (disque), ainsi que la finesse de maillage associée.

In [2]:
geo = SplineGeometry()

R = 1;
h = R/10; # ne pas prendre moins de R/50

pnts = [(R,0),(R,-R),(0,-R),(-R,-R),(-R,0),(-R,R),(0,R),(R,R)]

p = [geo.AppendPoint(*pnt) for pnt in pnts]


geo.Append(["spline3",*p[:3]],leftdomain=0,rightdomain=1,bc="bottom_right")
geo.Append(["spline3",*p[2:5]],leftdomain=0,rightdomain=1,bc="bottom_left")
geo.Append(["spline3",*p[4:7]],leftdomain=0,rightdomain=1,bc="up_left")
geo.Append(["spline3",*p[6:],p[0]],leftdomain=0,rightdomain=1,bc="up_right")

geo.SetMaterial(1, "in")

#geo.SetDomainMaxH(1, 0.05)
ngmesh = geo.GenerateMesh(maxh=h)
mesh = Mesh(ngmesh)
Draw (mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1, 'dr…

BaseWebGuiScene

## 2) Définition des problèmes éléments finis

L'objectif est ici de maximiser la différence de compliance magnétique pour deux chargement distincts. On va résoudre le problème magnétostatique dual ; pour cela, remarquons qu'en l'absence de termes sources:

$$ \overrightarrow{\text{rot}} ~ \overrightarrow{h} = 0 $$

On pose donc $\phi$ un potentiel scalaire tel que $\overrightarrow{h} = \overrightarrow{\text{grad}} ~ \phi$, qui vérifie donc dans $\Omega$:
$$ \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 $$

$\phi$ est un champ scalaire qu'on va supposer appartenant à $H^1$. Les conditions aux bords sont de Neumann. La solution est unique à une constante près. Les deux problèmes à résoudre s'écrivent donc :

$$\left \{
\begin{array}{l}
     \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 \\
     \frac{\partial \phi}{\partial n} = \Phi ~ \text{sur} ~ \Gamma_1 \bigcup \Gamma_2 \\
     \frac{\partial \phi}{\partial n} = - \Phi ~ \text{sur} ~ \Gamma_3 \bigcup \Gamma_4 
\end{array}
\right. $$

$$\left \{
\begin{array}{l}
     \text{div}( \mu ~\overrightarrow{\text{grad}} ~ \phi ) = 0 \\
     \frac{\partial \phi}{\partial n} = \Phi ~ \text{sur} ~ \Gamma_2 \bigcup \Gamma_3 \\
     \frac{\partial \phi}{\partial n} = - \Phi ~ \text{sur} ~ \Gamma_1 \bigcup \Gamma_4 
\end{array}
\right. $$


Sous forme faible, ce problème s'écrit donc :

$$\forall \psi \in H^1, \underbrace{\int_{\Omega} ~\overrightarrow{\text{grad}} ~ \psi \cdot  \mu ~\overrightarrow{\text{grad}} ~ \phi}_{\text{forme bilinéaire} } = \underbrace{\int_{\Gamma_+} \psi \Phi - \int_{\Gamma_-} \psi \Phi}_{\text{forme linéaire}} $$

Par ailleurs, on suppose que $\mu$ est constante par morceau.

In [3]:
PHI = 1;

fespace_mu = H1(mesh, order=0)
fespace_H1 = H1(mesh, order=1)
fespace_H1.FreeDofs()[0] = False

mu = GridFunction(fespace_mu)
mu.vec[:]=1


phi = fespace_H1.TrialFunction()
psi = fespace_H1.TestFunction()

K = BilinearForm(fespace_H1, symmetric=True)
K +=  InnerProduct(grad(psi),mu*grad(phi))*dx

n= specialcf.normal(mesh.dim);

l1 = LinearForm(fespace_H1)
l1 += InnerProduct(psi,-PHI)*ds(definedon=mesh.Boundaries("bottom_right|bottom_left"))
l1 += InnerProduct(psi,PHI)*ds(definedon=mesh.Boundaries("up_right|up_left"))

l2 = LinearForm(fespace_H1)
l2 += InnerProduct(psi,-PHI)*ds(definedon=mesh.Boundaries("bottom_right|up_right"))
l2 += InnerProduct(psi,PHI)*ds(definedon=mesh.Boundaries("bottom_left|up_left"))


In [4]:
K.Assemble()
l1.Assemble()
l2.Assemble()

In [5]:
H1 = GridFunction(fespace_H1)  # solution
H1.vec.data =     K.mat.Inverse(inverse="sparsecholesky") * l1.vec

H2 = GridFunction(fespace_H1)  # solution
H2.vec.data =     K.mat.Inverse(inverse="sparsecholesky") * l2.vec

In [7]:
Draw (H1, mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dr…

In [8]:
Draw (H2, mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2105-9-g5a835126f', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dr…